In [1]:
import pandas as pd
import requests
import time
import tqdm
import io
import json

In [2]:
path = 34
df_nombre = pd.read_csv(f'output/{path}_nombre_unique')
df_nombre = df_nombre.drop_duplicates(subset='ville')
display(df_nombre.describe(include='all'))

,Unnamed: 0,ville
count,24.000000,24
unique,NaN,24
top,NaN,Bedarieux
freq,NaN,1
mean,3413.041667,NaN
std,6154.271606,NaN
min,0.000000,NaN
25%,5.750000,NaN
50%,11.500000,NaN
75%,6332.500000,NaN


In [3]:
ajaccio = []
for i in df_nombre['ville']:
    reponse = requests.get(f"https://api-adresse.data.gouv.fr/search/?q={i}")
    reponse = reponse.json()
    ajaccio.append(reponse)
    time.sleep(60 / 300)


In [4]:
# Flatten the 'features' from each response in ajaccio and create a DataFrame
features_list = []
for response in ajaccio:
	if 'features' in response:
		features_list.extend(response['features'])

df_features = pd.json_normalize(features_list)
df_features.head()

,type,geometry.type,geometry.coordinates,properties.label,properties.score,properties.id,properties.banId,properties.type,properties.name,properties.postcode,...,properties.city,properties.context,properties.importance,properties.municipality,properties._type,properties.street,properties.district,properties.oldcitycode,properties.oldcity,properties.locality
0,Feature,Point,"[3.154586, 43.613706]",Bédarieux,0.943505,34028,767da111-2612-48ec-859c-28baed0cb033,municipality,Bédarieux,34600,...,Bédarieux,"34, Hérault, Occitanie",0.37855,Bédarieux,address,NaN,NaN,NaN,NaN,NaN
1,Feature,Point,"[3.223054, 43.357297]",Route de Bédarieux 34500 Béziers,0.700780,34032_1030,NaN,street,Route de Bédarieux,34500,...,Béziers,"34, Hérault, Occitanie",0.70858,NaN,address,Route de Bédarieux,NaN,NaN,NaN,NaN
2,Feature,Point,"[5.413322, 43.293706]",Impasse de bedarieux 13005 Marseille,0.694884,13205_0961,269544c8-d19b-49ca-aedd-38b962771d04,street,Impasse de bedarieux,13005,...,Marseille,"13, Bouches-du-Rhône, Provence-Alpes-Côte d'Azur",0.64372,NaN,address,Impasse de bedarieux,Marseille 5e Arrondissement,NaN,NaN,NaN
3,Feature,Point,"[3.162288, 43.612098]",Avenue Auguste Cot 34600 Bédarieux,0.692576,34028_0360,916eb808-3b95-413f-ae59-218ea90835be,street,Avenue Auguste Cot,34600,...,Bédarieux,"34, Hérault, Occitanie",0.61834,NaN,address,Avenue Auguste Cot,NaN,NaN,NaN,NaN
4,Feature,Point,"[3.152142, 43.61218]",Avenue Jean Jaures 34600 Bédarieux,0.692225,34028_0650,c17ca3a9-e977-4e02-a592-2fe47c1a59b9,street,Avenue Jean Jaures,34600,...,Bédarieux,"34, Hérault, Occitanie",0.61447,NaN,address,Avenue Jean Jaures,NaN,NaN,NaN,NaN


In [5]:
df_features[['longitude', 'latitude']] = pd.DataFrame(df_features['geometry.coordinates'].tolist(), index=df_features.index)

In [6]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   type                     237 non-null    object 
 1   geometry.type            237 non-null    object 
 2   geometry.coordinates     237 non-null    object 
 3   properties.label         237 non-null    object 
 4   properties.score         237 non-null    float64
 5   properties.id            237 non-null    object 
 6   properties.banId         196 non-null    object 
 7   properties.type          237 non-null    object 
 8   properties.name          237 non-null    object 
 9   properties.postcode      237 non-null    object 
 10  properties.citycode      237 non-null    object 
 11  properties.x             237 non-null    float64
 12  properties.y             237 non-null    float64
 13  properties.population    28 non-null     float64
 14  properties.city          2

In [7]:
df_features = df_features.drop(df_features.columns[[0,1,2,3,4,6,7,9,10,11,12,13,14,15,16,18,19,20,21]], axis=1)

In [8]:
df_features.head()

,properties.id,properties.name,properties.municipality,properties.oldcity,properties.locality,longitude,latitude
0,34028,Bédarieux,Bédarieux,NaN,NaN,3.154586,43.613706
1,34032_1030,Route de Bédarieux,NaN,NaN,NaN,3.223054,43.357297
2,13205_0961,Impasse de bedarieux,NaN,NaN,NaN,5.413322,43.293706
3,34028_0360,Avenue Auguste Cot,NaN,NaN,NaN,3.162288,43.612098
4,34028_0650,Avenue Jean Jaures,NaN,NaN,NaN,3.152142,43.612180


In [9]:
df_features['properties.municipality'].dropna()
df_features['properties.municipality'] = df_features['properties.municipality'].str.title()
df_features['properties.name'] = df_features['properties.name'].str.title()
df_features.head()

,properties.id,properties.name,properties.municipality,properties.oldcity,properties.locality,longitude,latitude
0,34028,Bédarieux,Bédarieux,NaN,NaN,3.154586,43.613706
1,34032_1030,Route De Bédarieux,NaN,NaN,NaN,3.223054,43.357297
2,13205_0961,Impasse De Bedarieux,NaN,NaN,NaN,5.413322,43.293706
3,34028_0360,Avenue Auguste Cot,NaN,NaN,NaN,3.162288,43.612098
4,34028_0650,Avenue Jean Jaures,NaN,NaN,NaN,3.152142,43.612180


In [10]:
df_filtre = df_features[df_features['properties.id'].str.startswith((f'{path}'))&
    (df_features['properties.id'].str.len() <= 5)]

In [11]:
display(df_filtre.describe(include='all'))

,properties.id,properties.name,properties.municipality,properties.oldcity,properties.locality,longitude,latitude
count,23,23,23,0,0,23.000000,23.000000
unique,23,23,23,0,0,NaN,NaN
top,34028,Bédarieux,Bédarieux,NaN,NaN,NaN,NaN
freq,1,1,1,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,3.337733,43.614993
std,NaN,NaN,NaN,NaN,NaN,0.320404,0.180560
min,NaN,NaN,NaN,NaN,NaN,2.675079,43.297541
25%,NaN,NaN,NaN,NaN,NaN,3.187432,43.479169
50%,NaN,NaN,NaN,NaN,NaN,3.317810,43.640760
75%,NaN,NaN,NaN,NaN,NaN,3.510205,43.729465


In [12]:
df_filtre.to_csv(f"output/{path}_new_gps.csv")

